Hämtar ut info om Medborgarförlag som finns i Wikidata med egenskap [Property:P9732](https://www.wikidata.org/wiki/Property:P9732)
* denna [Notebook](https://github.com/salgo60/medborgarforslag/blob/main/Jupyter/Medborgarf%C3%B6rslag%20WIkidata.ipynb)

In [10]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-09-05 21:52:31.126188


In [11]:
import sys,json
import pandas as pd 

from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?adminItem ?adminItemLabel ?lanidLabel ?url (SAMPLE(?coord) as ?coord) (SAMPLE(?img) as ?img) WHERE {
  ?adminItem wdt:P31 wd:Q127448;
    wdt:P17 wd:Q34. # Sverige
  OPTIONAL { ?adminItem wdt:P625 ?coord. }
  OPTIONAL { ?adminItem wdt:P856 ?www. }
  MINUS { ?adminItem wd:P576 ?end. } # kommuner som inte upphört
  MINUS { ?adminItem wd:P1366 ?sub. }
  { ?adminItem wdt:P9732 ?url}
  OPTIONAL { ?adminItem wdt:P41 ?flag }
  OPTIONAL { ?adminItem wdt:P94 ?coat }
  OPTIONAL { ?adminItem wdt:P154 ?logo }
  OPTIONAL { ?adminItem wdt:P18 ?image }
  ?adminItem wdt:P131 ?lanid.
  ?lanid wdt:P31 wd:Q200547.

  BIND( COALESCE(?flag, ?coat, ?logo, ?image) as ?img )
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
GROUP BY ?adminItem ?adminItemLabel  ?layer ?url ?lanidLabel ?www
order by ?lanidLabel"""

def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDMedborgar = get_sparql_dataframe(endpoint_url, query)

In [12]:
WDMedborgar.shape

(214, 6)

In [13]:
WDMedborgar.head(10)

,adminItem,adminItemLabel,lanidLabel,url,coord,img
0,http://www.wikidata.org/entity/Q504257,Olofströms kommun,Blekinge län,https://olofstrom.se/kommun-och-politik/politi...,Point(14.533333333 56.266666666),http://commons.wikimedia.org/wiki/Special:File...
1,http://www.wikidata.org/entity/Q510223,Karlshamns kommun,Blekinge län,https://www.karlshamn.se/kommun-och-politik/de...,Point(14.85 56.166666666),http://commons.wikimedia.org/wiki/Special:File...
2,http://www.wikidata.org/entity/Q515373,Ronneby kommun,Blekinge län,https://www.ronneby.se/kommun--politik/sa-arbe...,Point(15.283333333 56.2),http://commons.wikimedia.org/wiki/Special:File...
3,http://www.wikidata.org/entity/Q515409,Sölvesborgs kommun,Blekinge län,https://www.solvesborg.se/kommun-och-politik-/...,Point(14.583333333 56.05),http://commons.wikimedia.org/wiki/Special:File...
4,http://www.wikidata.org/entity/Q1128384,Karlskrona kommun,Blekinge län,https://www.karlskrona.se/kommun-och-politik/k...,Point(15.65 56.183333333),http://commons.wikimedia.org/wiki/Special:File...
5,http://www.wikidata.org/entity/Q504630,Orsa kommun,Dalarnas län,https://www.orsa.se/kommun-och-politik/politik...,Point(14.616666666 61.116666666),http://commons.wikimedia.org/wiki/Special:File...
6,http://www.wikidata.org/entity/Q505046,Smedjebackens kommun,Dalarnas län,https://www.smedjebacken.se/kommun-och-politik...,Point(15.416666666 60.133333333),http://commons.wikimedia.org/wiki/Special:File...
7,http://www.wikidata.org/entity/Q507684,Hedemora kommun,Dalarnas län,https://etjanster.hedemora.se/oversikt/overvie...,Point(15.983333333 60.283333333),http://commons.wikimedia.org/wiki/Special:File...
8,http://www.wikidata.org/entity/Q509651,Leksands kommun,Dalarnas län,https://www.leksand.se/medborgare/kommun-och-p...,Point(15.0 60.733333333),http://commons.wikimedia.org/wiki/Special:File...
9,http://www.wikidata.org/entity/Q1345439,Säters kommun,Dalarnas län,https://www.sater.se/kommun-demokrati/medborga...,Point(15.75 60.35),http://commons.wikimedia.org/wiki/Special:File...
